In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "prototyping"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol
using JLD2
using GLMakie
using GeometryBasics

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMolNotebooks
  ✓ MorphoMol
  ✓ MorphoMolNotebooks
  2 dependencies successfully precompiled in 12 seconds. 266 already precompiled.


In [2]:
Dict((1) => 0.1, (1,2) => 0.2)

Dict{Any, Float64} with 2 entries:
  (1, 2) => 0.2
  1      => 0.1

In [ ]:
function get_multichromatic_tetrahedra(points, n_atoms_per_mol)
    py"""
    import numpy as np
    import diode

    def get_multichromatic_tetrahedra(points, n_atoms_per_mol):        
        def is_multi(sigma):
            return len(set(v // n_atoms_per_mol for v in sigma)) >= 2
        points = np.asarray(points)
        tetrahedra = [vs for (vs,fs) in diode.fill_alpha_shapes(points) if len(vs) == 4 and is_multi(vs)]
        return tetrahedra
    """
    py"get_multichromatic_tetrahedra"(points, n_atoms_per_mol)
end

function get_barycenter(points, vertices) 
    Point3f(sum(points[vertices]) / length(vertices))
end

function get_barycentric_subdivision_and_filtration(points, mc_tets, n_atoms_per_mol::Int = 1206)
    barycenters = Vector{Point3f}([])
    filtration = Vector{Tuple{Vector{Int}, Float32}}([])
    new_simplices = Dict{Any, Float32}()
    already_converted_cells = Set{Vector{Int}}()
    total_vertices = 0
    for vs in eachrow(mc_tets)
        part_one = [v+1 for v in vs if div(v, n_atoms_per_mol)==0]
        part_two = [v+1 for v in vs if div(v, n_atoms_per_mol)==1]
        if length(part_one) == length(part_two) 
            #Get Simplices and Values for even split
            x,y = part_one[1], part_one[2]
            u,v = part_two[1], part_two[2]
            # potential_combinations = [[x,y,u,v], [x,v], [x,y,v], [y,v], [y,u,v], [y,u], [x,y,u], [x,u], [x,u,v]]

            # for comb in potential_combinations
            #     if !(comb in already_converted_cells)
            #         already_converted_cells = union(already_converted_cells, Set([comb]))
            #         barycenters = [barycenters; get_barycenter(points, comb)]
            # end
            barycenters = [barycenters; [
                get_barycenter(points, [x,y,u,v]),  #1
                get_barycenter(points, [x,v]),      #2
                get_barycenter(points, [x,y,v]),    #3
                get_barycenter(points, [y,v]),      #4
                get_barycenter(points, [y,u,v]),    #5
                get_barycenter(points, [y,u]),      #6
                get_barycenter(points, [x,y,u]),    #7
                get_barycenter(points, [x,u]),      #8
                get_barycenter(points, [x,u,v]),    #9
            ]]
            vertices = [
                ([1] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [u,v])) / 2.0), 
                ([2] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v])) / 2.0),
                ([3] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [v])) / 2.0),
                ([4] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [v])) / 2.0),
                ([5] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [u, v])) / 2.0),
                ([6] .+ total_vertices, euclidean(get_barycenter(points, [y]), get_barycenter(points, [u])) / 2.0),
                ([7] .+ total_vertices, euclidean(get_barycenter(points, [x,y]), get_barycenter(points, [u])) / 2.0),
                ([8] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u])) / 2.0),
                ([9] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v,u])) / 2.0),
            ]
            edges = [
                ([1,2] .+ total_vertices, minimum([vertices[1][2], vertices[2][2]])),
                ([1,3] .+ total_vertices, minimum([vertices[1][2], vertices[3][2]])),
                ([1,4] .+ total_vertices, minimum([vertices[1][2], vertices[4][2]])),
                ([1,5] .+ total_vertices, minimum([vertices[1][2], vertices[5][2]])),
                ([1,6] .+ total_vertices, minimum([vertices[1][2], vertices[6][2]])),
                ([1,7] .+ total_vertices, minimum([vertices[1][2], vertices[7][2]])),
                ([1,8] .+ total_vertices, minimum([vertices[1][2], vertices[8][2]])),
                ([1,9] .+ total_vertices, minimum([vertices[1][2], vertices[9][2]])),
                ([2,3] .+ total_vertices, minimum([vertices[2][2], vertices[3][2]])),
                ([3,4] .+ total_vertices, minimum([vertices[3][2], vertices[4][2]])),
                ([4,5] .+ total_vertices, minimum([vertices[4][2], vertices[5][2]])),
                ([5,6] .+ total_vertices, minimum([vertices[5][2], vertices[6][2]])),
                ([6,7] .+ total_vertices, minimum([vertices[6][2], vertices[7][2]])),
                ([7,8] .+ total_vertices, minimum([vertices[7][2], vertices[8][2]])),
                ([8,9] .+ total_vertices, minimum([vertices[8][2], vertices[9][2]])),
                ([9,2] .+ total_vertices, minimum([vertices[9][2], vertices[2][2]]))
            ]
            triangles = [
                ([1,2,3] .+ total_vertices, minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                ([1,3,4] .+ total_vertices, minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                ([1,4,5] .+ total_vertices, minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                ([1,5,6] .+ total_vertices, minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                ([1,6,7] .+ total_vertices, minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                ([1,7,8] .+ total_vertices, minimum([vertices[1][2], vertices[7][2], vertices[8][2]])),
                ([1,8,9] .+ total_vertices, minimum([vertices[1][2], vertices[8][2], vertices[9][2]])),
                ([1,9,2] .+ total_vertices, minimum([vertices[1][2], vertices[9][2], vertices[2][2]]))
            ]
            total_vertices += 9
            filtration = [filtration; vertices; edges; triangles]
        end
        if length(part_one) != length(part_two)
            if length(part_one) < length(part_two)
                part_one, part_two = part_two, part_one
            end
            u,v,w = part_one[1], part_one[2], part_one[3]
            x = part_two[1]

            barycenters = [barycenters; [
                get_barycenter(points, [u,v,w,x]),  #1
                get_barycenter(points, [x,v]),      #2
                get_barycenter(points, [x,v,w]),    #3
                get_barycenter(points, [x,w]),      #4
                get_barycenter(points, [w,u,x]),    #5
                get_barycenter(points, [u,x]),      #6
                get_barycenter(points, [u,v,x]),    #7
            ]]
            vertices = [
                ([1] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u,v,w])) / 2.0), 
                ([2] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v])) / 2.0),
                ([3] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [v,w])) / 2.0),
                ([4] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [w])) / 2.0),
                ([5] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [w,u])) / 2.0),
                ([6] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u])) / 2.0),
                ([7] .+ total_vertices, euclidean(get_barycenter(points, [x]), get_barycenter(points, [u,v])) / 2.0),
            ]
            edges = [
                ([1,2] .+ total_vertices, minimum([vertices[1][2], vertices[2][2]])),
                ([1,3] .+ total_vertices, minimum([vertices[1][2], vertices[3][2]])),
                ([1,4] .+ total_vertices, minimum([vertices[1][2], vertices[4][2]])),
                ([1,5] .+ total_vertices, minimum([vertices[1][2], vertices[5][2]])),
                ([1,6] .+ total_vertices, minimum([vertices[1][2], vertices[6][2]])),
                ([1,7] .+ total_vertices, minimum([vertices[1][2], vertices[7][2]])),
                ([2,3] .+ total_vertices, minimum([vertices[2][2], vertices[3][2]])),
                ([3,4] .+ total_vertices, minimum([vertices[3][2], vertices[4][2]])),
                ([4,5] .+ total_vertices, minimum([vertices[4][2], vertices[5][2]])),
                ([5,6] .+ total_vertices, minimum([vertices[5][2], vertices[6][2]])),
                ([6,7] .+ total_vertices, minimum([vertices[6][2], vertices[7][2]])),
                ([7,2] .+ total_vertices, minimum([vertices[7][2], vertices[2][2]]))
            ]
            triangles = [
                ([1,2,3] .+ total_vertices, minimum([vertices[1][2], vertices[2][2], vertices[3][2]])),
                ([1,3,4] .+ total_vertices, minimum([vertices[1][2], vertices[3][2], vertices[4][2]])),
                ([1,4,5] .+ total_vertices, minimum([vertices[1][2], vertices[4][2], vertices[5][2]])),
                ([1,5,6] .+ total_vertices, minimum([vertices[1][2], vertices[5][2], vertices[6][2]])),
                ([1,6,7] .+ total_vertices, minimum([vertices[1][2], vertices[6][2], vertices[7][2]])),
                ([1,7,2] .+ total_vertices, minimum([vertices[1][2], vertices[7][2], vertices[2][2]]))
            ]
            total_vertices += 7
            filtration = [filtration; vertices; edges; triangles]
        end
    end
    barycenters, filtration
end